In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

symbol = "NVDA"
period = '1y'
interval = '1h'

# 1 day = 7 hours from 14:30 - 20:30
df = yf.download(tickers=symbol, period=period, interval=interval)
df = df.reset_index()
df.columns = df.columns.get_level_values(0)


df.to_csv("NVDA.csv")
df.head()
# df["Close"]

[*********************100%***********************]  1 of 1 completed


Price,Datetime,Close,High,Low,Open,Volume
0,2024-01-04 14:30:00+00:00,47.954998,48.064003,47.507999,47.767002,8061891
1,2024-01-04 15:30:00+00:00,48.305660,48.401001,47.945000,47.947029,5778816
2,2024-01-04 16:30:00+00:00,48.269001,48.500000,48.215000,48.303009,4061918
3,2024-01-04 17:30:00+00:00,48.279999,48.289001,48.053001,48.268837,2479489
4,2024-01-04 18:30:00+00:00,48.144001,48.392002,48.123001,48.280998,2472828


In [2]:
import pandas_ta as ta
from scipy.signal import argrelextrema
import numpy as np



df['200EMA'] = ta.ema(df['Close'], length=200) # intermediate
ped_df = (df['Close'] - df['200EMA']) / df['Close']
df['Price_200EMA_diff'] = ped_df

adx = ta.adx(high=df['High'], low=df['Low'], close=df['Close'], length=14)
df['ADX'] = adx['ADX_14'] # intermediate

df['Short_MA'] = df['Close'].rolling(window=20).mean() # intermediate
df['Long_MA'] = df['Close'].rolling(window=50).mean() # intermediate

df['MA_CO_signal'] = 0
df.loc[(df['Short_MA'] > df['Long_MA']) & (df['ADX'] > 25), 'MA_CO_signal'] = 1  
df.loc[(df['Short_MA'] <= df['Long_MA']) & (df['ADX'] > 25), 'MA_CO_signal'] = -1  



# order = 5  
# df['Swing_High'] = df.iloc[argrelextrema(df['High'].values, np.greater, order=order)[0]]['High'] # intermediate
# df['Swing_Low'] = df.iloc[argrelextrema(df['Low'].values, np.less, order=order)[0]]['Low'] # intermediate
# df['Current_Support'] = np.nan # intermediate
# df['Current_Resistance'] = np.nan # intermediate
# current_support = np.nan
# current_resistance = np.nan
# tolerance = 0.01

# for idx, row in df.iterrows():
#     if not np.isnan(row['Swing_Low']):
#         current_support = row['Swing_Low']
#     if not np.isnan(row['Swing_High']):
#         current_resistance = row['Swing_High']
    
#     df.at[idx, 'Current_Support'] = current_support
#     df.at[idx, 'Current_Resistance'] = current_resistance

# df['Close_Support_Diff'] = (df['Close'] - df['Current_Support']) / df['Current_Support']  # intermediate
# df['Close_Resistance_Diff'] = (df['Close'] - df['Current_Resistance']) / df['Current_Resistance']  # intermediate

# df['On_Support'] = df['Close_Support_Diff'].abs() <= tolerance # intermediate
# df['On_Resistance'] = df['Close_Resistance_Diff'].abs() <= tolerance # intermediate 

# df['Support_Lag1'] = df['Current_Support'].shift(1)
# df['Resistance_Lag1'] = df['Current_Resistance'].shift(1)
# df['Distance_to_Support'] = df['Close_Support_Diff']
# df['Distance_to_Resistance'] = df['Close_Resistance_Diff']

# df.drop(['Current_Support'])
# df.drop(['Current_Resistance'])

df


Price,Datetime,Close,High,Low,Open,Volume,200EMA,Price_200EMA_diff,ADX,Short_MA,Long_MA,MA_CO_signal
0,2024-01-04 14:30:00+00:00,47.954998,48.064003,47.507999,47.767002,8061891,NaN,NaN,NaN,NaN,NaN,0
1,2024-01-04 15:30:00+00:00,48.305660,48.401001,47.945000,47.947029,5778816,NaN,NaN,NaN,NaN,NaN,0
2,2024-01-04 16:30:00+00:00,48.269001,48.500000,48.215000,48.303009,4061918,NaN,NaN,NaN,NaN,NaN,0
3,2024-01-04 17:30:00+00:00,48.279999,48.289001,48.053001,48.268837,2479489,NaN,NaN,NaN,NaN,NaN,0
4,2024-01-04 18:30:00+00:00,48.144001,48.392002,48.123001,48.280998,2472828,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1747,2025-01-03 16:30:00+00:00,143.680298,144.130005,143.180099,143.470001,13728139,137.557765,0.042612,23.655643,137.649194,137.964663,0
1748,2025-01-03 17:30:00+00:00,144.495300,144.740005,143.639999,143.690002,14507644,137.626795,0.047534,25.073163,137.923409,138.176771,-1
1749,2025-01-03 18:30:00+00:00,144.371002,144.899994,143.820007,144.500000,14080568,137.693902,0.046250,26.440371,138.211964,138.364991,-1
1750,2025-01-03 19:30:00+00:00,144.320007,144.550003,144.039993,144.369995,11049517,137.759833,0.045456,27.709922,138.557464,138.527391,1


In [3]:
# Count rows with at least one NaN
num_rows_with_nan = df.isna().any(axis=1).sum()

print(f"Number of rows with at least one NaN: {num_rows_with_nan}")

df.shape

Number of rows with at least one NaN: 199


(1752, 12)

In [4]:
df.dropna(inplace=True)
df = df.iloc[4:] # now data starts from 2024-02-15, and each day has 7 rows from 14:30 - 20:30 inclusive
df.shape


(1549, 12)

In [5]:
df.to_csv("NVDA_cleaned.csv")
df.shape

(1549, 12)